In [1]:
!pip install transformers
!pip install tensorflow

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 4.2 MB 642 kB/s eta 0:00:01
     |████████████████████████████████| 764 kB 575 kB/s eta 0:00:01
     |████████████████████████████████| 701 kB 629 kB/s eta 0:00:01
     |████████████████████████████████| 6.6 MB 644 kB/s eta 0:00:01
     |████████████████████████████████| 84 kB 640 kB/s eta 0:00:01
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 511.7 MB 579 kB/s eta 0:00:01     |███████████████████████████▏    | 434.7 MB 571 kB/s eta 0:02:15     |███████████████████████████████ | 496.1 MB 567 kB/s eta 0:00:28
     |████████████████████████████████| 57 kB 462 kB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 580 kB/s eta 0:00:01
     |████████████████████████████████| 4.5 MB 605 kB/s eta 0:00:01     |██████████████████▉             | 2.6 MB 597 kB/s eta 0:00:04
     |████████████████████████████████| 5.8 MB 587 kB/s eta 0:00:01

In [1]:
import torch
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

In [2]:
import torch
import csv
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from PIL import Image, ImageFile
import torchvision.transforms as transforms
from torchvision import models
import torch.utils.model_zoo as model_zoo
import torch.nn.functional as F
import torch.optim as optim
import os
import json
from transformers import BertTokenizer, BertModel
from transformers import XLNetModel, XLNetTokenizerFast
import operator

2022-05-22 15:29:07.760183: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
device = torch.device('cuda')
tokenizer = XLNetTokenizerFast.from_pretrained("xlnet-base-cased")
pre_trained_model = XLNetModel.from_pretrained("xlnet-base-cased")

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
class ContextQuType:
    def __init__(self, tokenizer, length, ids, context, question):
        self.tokenizer = tokenizer
        self.codeLength = length
        self.ids = ids
        self.context = context
        self.question = question
        self.after_encode = 0
        
    def convert(self):
        if self.after_encode==0:
            after_encode = tokenizer(self.question,self.context,max_length=self.codeLength,padding="max_length")
        return torch.stack([torch.tensor(after_encode['input_ids'][-512:]),torch.tensor(after_encode['token_type_ids'][-512:]),torch.tensor(after_encode['attention_mask'][-512:])])
        
class LabelType:
    def __init__(self, tokenizer, length, CQu, has_ans, text, start):
        self.tokenizer = tokenizer
        self.codeLength = length
        self.has_ans = has_ans
        self.text = text
        self.start = start
        self.CQu = CQu
    
    def convert(self):
        # make groundtruth
        S = [0.0] * self.codeLength
        E = [0.0] * self.codeLength
        background = tokenizer.encode(self.CQu.question,self.CQu.context,max_length=self.codeLength,padding="max_length")
        length_exceed = False
        if len(background) > self.codeLength:
            length_exceed = True
        if self.has_ans:
            after_encode = tokenizer.encode(self.text)[:-2]
            ans_length = len(after_encode)
            after_answer = tokenizer.encode(self.CQu.context[self.start:])
            if length_exceed:
                start_id = len(background) - len(after_answer)
                end_id = start_id + ans_length -1
                if tokenizer.decode(background[start_id:end_id+1])!=self.text:
                    lower = max(start_id-3,0)
                    upper = min(end_id+4,len(background))
                    ans_found = False
                    for i in range(lower,upper):
                        for j in range(i+1,upper):
                            if tokenizer.decode(background[i:j])==self.text:
                                start_id = i
                                end_id = j-1
                                ans_found = True
                                break
                            if ans_found:
                                break
                if end_id >= self.codeLength:
                    end_id = self.codeLength-1
                if start_id >= self.codeLength:
                    start_id = 0
                    end_id = 0
                S[start_id] = 1.0
                E[end_id] = 1.0
            else:
                start_id = len(background) - len(after_answer)
                end_id = start_id + ans_length -1
                if tokenizer.decode(background[start_id:end_id+1])!=self.text:
                    lower = max(start_id-3,0)
                    upper = min(end_id+4,len(background))
                    ans_found = False
                    for i in range(lower,upper):
                        for j in range(i+1,upper):
                            candidate = tokenizer.decode(background[i:j])
                            if len(candidate)>0:
                                if candidate[0]=='$' and self.text[0]!='$':
                                    candidate=candidate[1:]
                                if candidate[0]!='$' and self.text[0]=='$':
                                    candidate='$'+candidate
                            if candidate==self.text:
                                start_id = i
                                end_id = j-1
                                ans_found = True
                                break
                            if ans_found:
                                break
                S[start_id] = 1.0
                E[end_id] = 1.0
            #if tokenizer.decode(background[start_id:end_id+1])!=self.text:
                #print(tokenizer.convert_ids_to_tokens(background))
                #print(tokenizer.decode(after_answer[:ans_length])+'|||'+tokenizer.decode(background[start_id:end_id+1])+'|||'+self.text)
                #print(len(background),len(after_answer),start_id,end_id)
                #print(self.CQu.context[self.start:self.start+len(self.text)]+'|||'+tokenizer.decode(after_encode)+'|||',tokenizer.convert_ids_to_tokens(after_encode))
        else:
            S[-1] = 1.0
            E[-1] = 1.0
        return torch.stack([torch.tensor(S),torch.tensor(E)])

In [6]:
def load_data(load_path,tokenizer,length):
    with open(load_path) as json_data:
        dev = json.load(json_data)['data']
    CQu = []
    Label = []
    for data in dev:
        for paragraphs in data['paragraphs']:
            context = paragraphs['context']
            for qas in paragraphs['qas']:
                question = qas['question']
                ids = qas['id']
                ctx = ContextQuType(tokenizer,length,ids,context,question)
                CQu.append(ctx)
                if qas['is_impossible']:
                    Label.append(LabelType(tokenizer,length,ctx,False,'',0))
                else:
                    answer = qas['answers'][0]
                    Label.append(LabelType(tokenizer,length,ctx,True,answer['text'],answer['answer_start']))
    return CQu,Label

In [7]:
class OutputPair:
    def __init__(self,logits_start,logits_end):
        self.logits_start = logits_start
        self.logits_end = logits_end

class CrossEntropy(nn.Module):
    def __init__(self):
        super(CrossEntropy, self).__init__()
    def forward(self,outputs,labels):
        log_soft = F.log_softmax(outputs.logits_start,dim=1)
        loss_start = F.cross_entropy(log_soft, labels[:,0])
        loss_end   = F.cross_entropy(F.log_softmax(outputs.logits_end,dim=1), labels[:,1])
        return loss_start + loss_end / 2.0

class BERTQuA(nn.Module):
    def __init__(self, bert, tokenizer,code_length,device,epsilon = 1.0):
        super(BERTQuA, self).__init__()
        self.tokenizer = tokenizer
        self.device = device
        self.encoder = bert.to(device)
        self.code_length = code_length
        self.output_start = nn.Linear(768,1).to(device)
        self.output_end = nn.Linear(768,1).to(device)
        self.epsilon = 1.0

    def forward(self, inputs):
        tokens_X, segments_X, masks = inputs[:,0],inputs[:,1],inputs[:,2]
        encoded_X = self.encoder(tokens_X, token_type_ids=segments_X, attention_mask=masks).last_hidden_state
        return OutputPair(self.output_start(encoded_X),self.output_end(encoded_X))
        
    def prediction(self,question,answer_text):
        ctx = ContextQuType(self.tokenizer,self.code_length,'',answer_text,question)
        inputs = torch.stack([ctx.convert()]).to(device)
        token_ids = tokenizer(question,answer_text,max_length=self.code_length,padding="max_length")['input_ids']
        outputs = self.forward(inputs)
        start_scores = outputs.logits_start.reshape(-1)
        end_scores = outputs.logits_end.reshape(-1)
        no_ans_scores = start_scores[-1] + end_scores[-1]
        answer = ''
        tokens = tokenizer.convert_ids_to_tokens(token_ids)
        #print(outputs.logits_start.reshape(-1))
        #print(outputs.logits_end.reshape(-1))
        if (torch.max(start_scores) + torch.max(end_scores) <= no_ans_scores +  self.epsilon):
            answer = ''
        else:
            add_space = False
            answer_start = torch.argmax(start_scores)
            answer_end = torch.argmax(end_scores)
            answer = self.tokenizer.decode(token_ids[answer_start:answer_end+1])
        return answer
    def predictionLabel(self,label):
        #ctx = ContextQuType(self.tokenizer,self.code_length,'',answer_text,question)
        #inputs = torch.stack([ctx.convert()]).to(device)
        #token_ids = tokenizer(question,answer_text,max_length=self.code_length,padding="max_length")['input_ids']
        #outputs = self.forward(inputs)
        outputs = label.convert()
        #print(outputs)
        token_ids = tokenizer(label.CQu.question,label.CQu.context,max_length=self.code_length,padding="max_length")['input_ids']
        start_scores = outputs[0,:]
        end_scores = outputs[1,:]
        #print(start_scores,end_scores)
        no_ans_scores = start_scores[-1] + end_scores[-1]
        answer = ''
        tokens = tokenizer.convert_ids_to_tokens(token_ids)
        #print(outputs.logits_start.reshape(-1))
        #print(outputs.logits_end.reshape(-1))
        if (torch.max(start_scores) + torch.max(end_scores) <= no_ans_scores +  self.epsilon):
            answer = ''
        else:
            answer_start = torch.argmax(start_scores)
            answer_end = torch.argmax(end_scores)
            answer = self.tokenizer.decode(token_ids[answer_start:answer_end+1])
        return answer
    def load_data(self,file_path):
        CQu, La = load_data(file_path,self.tokenizer,self.code_length)
        self.inputs = torch.stack([ctx.convert() for ctx in CQu])
        self.labels = torch.stack([ctx.convert() for ctx in La])
    
    def train(self, epochs=3, batch_size = 48, lr = 3e-5, T=10):
        optimizer = optim.Adam(self.parameters(), lr=lr)
        criterion = CrossEntropy().to(device)
        self.train_loader = DataLoader(TensorDataset(self.inputs,self.labels),batch_size = batch_size, shuffle = True, num_workers = 2)
        epoch_loss = []
        for epoch in range(epochs):      
            running_loss = 0.0
            epoch_running_loss = 0.0
            batch_count = 0
            for batchidx, (x, label) in enumerate(self.train_loader):
                x, label = x.to(device), label.to(device)
                output = self.forward(x)
                output.logits_start=output.logits_start.resize(len(output.logits_start),self.code_length)
                output.logits_end=output.logits_end.resize(len(output.logits_end),self.code_length)
                loss = criterion(output, label)
                # backprop
                optimizer.zero_grad()  #梯度清0
                loss.backward()   #梯度反传
                optimizer.step()   #保留梯度

                running_loss += loss.item()
                epoch_running_loss += loss.item()
                batch_count += 1
                if batchidx % T == T-1:
                    print(epoch,' batchidx: ', batchidx, ' loss: ', running_loss/T)
                    running_loss = 0.0
            epoch_loss.append(epoch_running_loss/batch_count)
            print(epoch, 'loss:', epoch_running_loss/batch_count)
        return epoch_loss
    def printLabel(self,file_path):
        CQu, La = load_data(file_path,self.tokenizer,self.code_length)
        count = 0
        precise = 0
        for label in La:
            count += 1
            predict = self.predictionLabel(label)
            if predict==label.text:
                precise += 1
            #else:
            #    print(predict,label.text,label.has_ans)
        print(precise/count)

In [7]:
device = torch.device('cuda')
net = BERTQuA(pre_trained_model,tokenizer,512,device)
net.load_data('classifySimBertCluster2.json')
net.printLabel('classifySimBertCluster2.json')

0.9840725746875092


In [8]:
epoch_loss = net.train(batch_size=16,epochs=3)
f = open("loss_cluster2.txt", "w")
f.write(str(epoch_loss))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/root/miniconda3/lib/python3.8/site-packages/torch/_tensor.py:493: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


0  batchidx:  9  loss:  7.832815790176392
0  batchidx:  19  loss:  6.6013837337493895
0  batchidx:  29  loss:  5.789677381515503
0  batchidx:  39  loss:  5.4132115840911865
0  batchidx:  49  loss:  5.117201662063598
0  batchidx:  59  loss:  4.79096667766571
0  batchidx:  69  loss:  3.9224693775177
0  batchidx:  79  loss:  3.638869047164917
0  batchidx:  89  loss:  3.5030776977539064
0  batchidx:  99  loss:  3.11249897480011
0  batchidx:  109  loss:  2.8518182277679442
0  batchidx:  119  loss:  3.2334677815437316
0  batchidx:  129  loss:  2.6879336833953857
0  batchidx:  139  loss:  2.994974064826965
0  batchidx:  149  loss:  2.6096715688705445
0  batchidx:  159  loss:  2.764068031311035
0  batchidx:  169  loss:  2.819508504867554
0  batchidx:  179  loss:  2.853278648853302
0  batchidx:  189  loss:  2.9397136211395263
0  batchidx:  199  loss:  2.7830420255661013
0  batchidx:  209  loss:  2.6861269235610963
0  batchidx:  219  loss:  3.2049712419509886
0  batchidx:  229  loss:  2.63894450

60

In [9]:
question = """Which journalist considered Spectre the worst James Bond movie in three decades?"""
context = """Critical appraisal of the film was mixed in the United States. In a lukewarm review for RogerEbert.com, Matt Zoller Seitz gave the film 2.5 stars out of 4, describing Spectre as inconsistent and unable to capitalise on its potential. Kenneth Turan, reviewing the film for Los Angeles Times, concluded that Spectre \"comes off as exhausted and uninspired\". Manohla Dargis of The New York Times panned the film as having \"nothing surprising\" and sacrificing its originality for the sake of box office returns. Forbes' Scott Mendelson also heavily criticised the film, denouncing Spectre as \"the worst 007 movie in 30 years\". Darren Franich of Entertainment Weekly viewed Spectre as \"an overreaction to our current blockbuster moment\", aspiring \"to be a serialized sequel\" and proving \"itself as a Saga\". While noting that \"[n]othing that happens in Spectre holds up to even minor logical scrutiny\", he had \"come not to bury Spectre, but to weirdly praise it. Because the final act of the movie is so strange, so willfully obtuse, that it deserves extra attention.\" In a positive review Rolling Stone, Peter Travers gave the film 3.5 stars out of 4, describing \"The 24th movie about the British MI6 agent with a license to kill is party time for Bond fans, a fierce, funny, gorgeously produced valentine to the longest-running franchise in movies\". Other positive reviews from Mick LaSalle from the San Francisco Chronicle, gave it a perfect 100 score, stating: \u201cOne of the great satisfactions of Spectre is that, in addition to all the stirring action, and all the timely references to a secret organization out to steal everyone\u2019s personal information, we get to believe in Bond as a person.\u201d Stephen Whitty from the New York Daily News, gave it an 80 grade, saying: \u201cCraig is cruelly efficient. Dave Bautista makes a good, Oddjob-like assassin. And while Lea Seydoux doesn\u2019t leave a huge impression as this film\u2019s \u201cBond girl,\u201d perhaps it\u2019s because we\u2019ve already met \u2014 far too briefly \u2014 the hypnotic Monica Bellucci, as the first real \u201cBond woman\u201d since Diana Rigg.\u201d Richard Roeper from the Chicago Sun-Times, gave it a 75 grade. He stated: \u201cThis is the 24th Bond film and it ranks solidly in the middle of the all-time rankings, which means it\u2019s still a slick, beautifully photographed, action-packed, international thriller with a number of wonderfully, ludicrously entertaining set pieces, a sprinkling of dry wit, myriad gorgeous women and a classic psycho-villain who is clearly out of his mind but seems to like it that way.\u201d Michael Phillips over at the Chicago Tribune, gave it a 75 grade. He stated: \u201cFor all its workmanlike devotion to out-of-control helicopters, \u201cSpectre\u201d works best when everyone\u2019s on the ground, doing his or her job, driving expensive fast cars heedlessly, detonating the occasional wisecrack, enjoying themselves and their beautiful clothes.\u201d Guy Lodge from Variety, gave it a 70 score, stating: \u201cWhat\u2019s missing is the unexpected emotional urgency of \u201cSkyfall,\u201d as the film sustains its predecessor\u2019s nostalgia kick with a less sentimental bent.\u201d"""
net.prediction(question,context)

''

In [10]:
torch.save({'state_dict': net.state_dict(), 'epoch':3},'xlnetCluster2'+str(3) + '.pkl')